# Create normalized 3 channel images (nucleus, membrane, & CD34)

#### This notebook normalized the deconvolved nuclear, membrane (B2M/CD298), and CD34 channels to create a pseudo-colored image for segmentation and visualization. Code provided here normalized the images from patient P56 but analogous code is used for all pateints.

In [1]:
import numpy as np
import warnings
import os
import napari
import tifffile
import cv2 as cv
import pandas as pd
from pathlib import Path

## PNG_Setting

In [2]:
# whether to generate PNG

PNG_GENERATE = True

## Def list

In [3]:
# normalize image (to 16-bit)
def image_normalization(image, img_min=0, img_max=65535):
    image = np.float32(image)
    epsilon = 1e-12
    image = (image - np.min(image)) * (img_max - img_min) / ((np.max(image) - np.min(image)) + epsilon) + img_min
    return np.uint16(image)

# set-up napari viewer used to combine channels into single pseudo-colored image   
def set_viewer(unset_view, contrast_in = None, colormap=None):
    if contrast_in is None:
        contrast_in = [[0, 65535],[0, 65535],[0, 65535]] # only retaining DAPI, CD298/B2M, and CD34
    if colormap is None:
        colormap = ['green','magenta','blue'] # define colors to use for each channel
        
    unset_view.layers['Image'].name = 'CD298'
    unset_view.layers['Image [1]'].name = 'CD34'
    unset_view.layers['Image [2]'].name = 'DAPI'

    unset_view.layers['CD298'].contrast_limits = contrast_in[0]
    unset_view.layers['CD34'].contrast_limits = contrast_in[1]
    unset_view.layers['DAPI'].contrast_limits = contrast_in[2]

    unset_view.layers['CD298'].colormap = colormap[0]
    unset_view.layers['CD34'].colormap = colormap[1]
    unset_view.layers['DAPI'].colormap = colormap[2]

In [4]:
# list of all FOV name extensions for given patient
fovs = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']

## Input directory

In [5]:
# assign directory
warnings.filterwarnings('ignore')
default_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/0_RawImages/28_110038_S2/'
os.chdir(default_dir)
cwd = os.getcwd()

## Output 3 channel png

In [7]:
# Read in deconvolved membrane, CD34, and dapi channels

for i in range(len(fovs)):
    
    # define base filename
    FileName = '20220228_110038_S2_C902_P99_N99_F0' + fovs[i]

    # Read all deconvolved channels
    
    ch0 = [];
    ch1 = [];
    ch2 = [];

    # read in each z-slice for 3 channels of interest
    for j in range(9):

        # read tif files
        img0 = tifffile.imread(FileName + '.ome_cmle_z00' + str(j) + '_ch00.tif')
        ch0.append(img0)
        img1 = tifffile.imread(FileName + '.ome_cmle_z00' + str(j) + '_ch03.tif')
        ch1.append(img1)
        img2 = tifffile.imread(FileName + '.ome_cmle_z00' + str(j) + '_ch04.tif')
        ch2.append(img2)

    # make z-stack for each channel
    all_ch0 = np.stack(ch0, axis=0)
    all_ch1 = np.stack(ch1, axis=0)
    all_ch2 = np.stack(ch2, axis=0)


    # z-projection - take the average value of each z layer
    stack_ch0 = np.sum(all_ch0, axis=0, dtype=int) / all_ch0.shape[0]
    stack_ch1 = np.sum(all_ch1, axis=0, dtype=int) / all_ch1.shape[0]
    stack_ch2 = np.sum(all_ch2, axis=0, dtype=int) / all_ch2.shape[0]

    
    # combine all channels in one matrix
    img = np.stack((stack_ch0, stack_ch1, stack_ch2), axis=0)
    
    # find normalization cutoff pixel values for nuclear, membrane, and CD34 channels
    CD298 = img[0, :, :]
    CD298_low = np.percentile(CD298.flatten(), 5)
    CD298_high = np.percentile(CD298.flatten(), 99.99)
    
    CD34 = img[1, :, :]
    CD34_low = np.percentile(CD34.flatten(), 5)
    CD34_high = np.percentile(CD34.flatten(), 99.99)
    
    nuclei = img[2, :, :]
    nuc_low = np.percentile(nuclei.flatten(), 5)
    nuc_high = np.percentile(nuclei.flatten(), 99.99)

    
    
    # call set viewer function
    viewer = napari.view_image(img, channel_axis=0, show=False)
    set_viewer(viewer)
    napari.run()
    
    # set contrast and color
    contrast = [[CD298_low, CD298_high], [CD34_low, CD34_high], [nuc_low, nuc_high]]
    color = [viewer.layers['CD298'].colormap, viewer.layers['CD34'].colormap, viewer.layers['DAPI'].colormap]
    num = 0
    
    img = img.astype('float64')
    
    try:
        del img
    finally:
        if PNG_GENERATE:
            img_tif_raw = img
            viewer_1 = napari.view_image(img_tif_raw, channel_axis=0, show=False)
            set_viewer(viewer_1, contrast, color)
            viewer_1.scale_bar.visible = True
            viewer_1.scale_bar.unit = "pixel(0.18 micron/pixel)"
            blended = np.zeros(viewer_1.layers[0].data.shape + (4,))
            for layer in viewer_1.layers:
                # normalize data by clims
                layer.data[layer.data > layer.contrast_limits[1]] = layer.contrast_limits[1]
                layer.data[layer.data < layer.contrast_limits[0]] = layer.contrast_limits[0]
                normalized_data = (layer.data - layer.contrast_limits[0]) / (
                        layer.contrast_limits[1] - layer.contrast_limits[0])
                normalized_flatdata = normalized_data.flatten()
                normalized_flatdata[normalized_flatdata < 0] = 0
                colormapped_data = layer.colormap.map(normalized_flatdata)
                colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))
                blended = blended + colormapped_data
            blended[..., 3] = 1  # set alpha channel to 1
            img_png = image_normalization(blended)
            img_png = img_png[:, :, (2, 1, 0)]            
            cv.imwrite('/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized_DAPI_B2M_CD34/' + FileName + '_normalized.png', img_png, [cv.IMWRITE_PNG_COMPRESSION, 0])

        num = num + 1
        del img_png, blended, viewer_1
    